# 1. gradio로 애플, 마이크로소프트, 테슬라, 엔비디아 주식 - yahoo finance에서 2014-전날까지 불러오기
# 2. 데이터 표시
# 3. 종가를 타겟 변수로 지정
# 4. AutoGluon을 이용해 시계열 분석
# 5. 30일 예측 결과 출력, 시각화

In [1]:
import pandas as pd
import numpy as np
import gradio as gr
import yfinance as yf
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

In [2]:
import yfinance as yf
import pandas as pd
import gradio as gr
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# 종목 리스트
tickers = ["AAPL", "MSFT", "TSLA", "NVDA"]

# 1. Yahoo Finance에서 주식 데이터를 불러오는 함수
def load_stock_data(ticker):
    end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    stock_data = yf.download(ticker, start="2014-01-01", end=end_date)
    if stock_data.empty:
        raise ValueError(f"선택된 주식의 데이터가 없습니다: {ticker}.")
    return stock_data

# 2. 데이터프레임에서 필요한 정보(최근 데이터)를 표시하는 함수
def display_data(stock_data):
    return stock_data.tail()

# 3. AutoGluon을 사용하여 30일 시계열 예측을 수행하는 함수
def time_series_forecast(stock_data, days=30, time_limit=300):
    stock_data = stock_data.reset_index()
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    
    # 열 이름 재정의
    stock_data = stock_data[['Date', 'Close']]
    stock_data.columns = ['datetime', 'target']  # AutoGluon 호환성을 위해 열 이름 변경

    # 종목 ID 추가 (AutoGluon에서 고유 ID가 필요함)
    stock_data['item_id'] = 0  # 하나의 시계열이므로 ID를 동일하게 설정

    # 데이터를 TimeSeriesDataFrame 형식으로 변환
    ts_data = TimeSeriesDataFrame.from_data_frame(
        stock_data, 
        id_column="item_id", 
        timestamp_column="datetime"
    )

    # 일별 빈도로 데이터 변환
    ts_data = ts_data.convert_frequency(freq='D')

    # AutoGluon 예측 모델 초기화
    predictor = TimeSeriesPredictor(
        prediction_length=days,
        freq='D'  # 일별 빈도
    )
    
    # 모델 학습
    predictor.fit(train_data=ts_data)

    # 미래 30일 예측 수행
    forecast = predictor.predict(ts_data)
    
    return forecast

# 4. 예측 결과와 그래프를 시각화하는 함수
def plot_forecast(stock_data, forecast):
    plt.figure(figsize=(10, 6))
    
    # 실제 주가 데이터 시각화
    plt.plot(stock_data['datetime'], stock_data['target'], label="실제 주가")
    
    # 예측된 주가 데이터 시각화
    plt.plot(forecast.index, forecast['mean'], label="예측된 주가", linestyle="--")
    
    plt.title("주가 예측")
    plt.xlabel("날짜")
    plt.ylabel("가격")
    plt.legend()
    plt.grid(True)
    
    # 그래프를 파일로 저장
    plt.savefig("forecast_plot.png")
    
    return "forecast_plot.png"

# Gradio UI 정의
def predict_stock(ticker):
    try:
        # 주식 데이터를 불러오고 필요한 정보 표시
        stock_data = load_stock_data(ticker)
        displayed_data = display_data(stock_data)
        
        # 시계열 예측 수행 및 결과 시각화
        forecast = time_series_forecast(stock_data)
        plot_path = plot_forecast(stock_data, forecast)
    
        return displayed_data, plot_path
    except Exception as e:
        # 오류 발생 시 메시지 반환
        error_df = pd.DataFrame({"오류": [str(e)]})
        return error_df, None

# Gradio 인터페이스 설정
interface = gr.Interface(
    fn=predict_stock, 
    inputs=gr.Dropdown(choices=tickers, label="주식을 선택하세요", value="AAPL"),  # 선택 가능한 주식 리스트
    outputs=[gr.Dataframe(label="주식 데이터"), gr.Image(label="예측 그래프")],
    title="주가 예측 시스템",
    description="AutoGluon을 사용해 Apple, Microsoft, Tesla, Nvidia의 30일 미래 주가 예측."
)

# Gradio 앱 실행
interface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


[*********************100%***********************]  1 of 1 completed
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241021_003414'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          12
GPU Count:          0
Memory Avail:       12.25 GB / 15.31 GB (80.0%)
Disk Space Avail:   39.00 GB / 237.85 GB (16.4%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'D',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 30,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

Provided train_data has 3943 rows (NaN fraction=31.1%), 1 time serie

In [113]:
interface.close()

Closing server running on port: 7865
